In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importing libraries

In [17]:
import os
import shutil
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report

In [18]:
# Downloaded data from "https://www.kaggle.com/datasets/roudranildas/chicken-images-classification-dataset"

# Preparing folders for ImageFolder
root = 'dataset'
categories = ['chicken-images', 'duck-images']
splits = ['train', 'val', 'test']

for split in splits:
    for folder_name in categories:
        src = f'/content/drive/MyDrive/CMI/data/{folder_name}/data/{split}'
        dest = f'/content/drive/MyDrive/CMI/{root}/{split}/{folder_name.split("-")[0]}'
        os.makedirs(dest, exist_ok=True)

        for img_file in os.listdir(src):
            full_path = os.path.join(src, img_file)
            if os.path.isfile(full_path):
                shutil.copy(full_path, dest)

In [19]:
%cd /content/drive/MyDrive/CMI/

/content/drive/MyDrive/CMI


In [20]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train = datasets.ImageFolder('dataset/train', transform = transform)
val = datasets.ImageFolder('dataset/val', transform = transform)
test = datasets.ImageFolder('dataset/test', transform = transform)

train_loader = DataLoader(train, batch_size = 32, shuffle=True)
val_loader = DataLoader(val, batch_size = 32, shuffle=False)
test_loader = DataLoader(test, batch_size = 32, shuffle=False)

class_names = train.classes
print("Classes:", class_names)

Classes: ['chicken', 'duck']


In [21]:
# Setting up the device
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print("Device:", device)

Device: cpu


# Finetuning the model using Resnet18

In [22]:
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr = 0.001)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Classification report before training

In [23]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        preds = torch.argmax(outputs, 1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

print(classification_report(all_labels, all_preds, target_names=class_names))

              precision    recall  f1-score   support

     chicken       0.21      0.20      0.21       172
        duck       0.56      0.57      0.57       310

    accuracy                           0.44       482
   macro avg       0.38      0.39      0.39       482
weighted avg       0.44      0.44      0.44       482



In [24]:
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

Epoch 1/5, Loss: 0.4089
Epoch 2/5, Loss: 0.1922
Epoch 3/5, Loss: 0.1660
Epoch 4/5, Loss: 0.1476
Epoch 5/5, Loss: 0.2013


In [25]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        preds = torch.argmax(outputs, 1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

## Classification report after training

In [26]:
print("Classification report: ")
print(classification_report(all_labels, all_preds, target_names=class_names))

Classification report: 
              precision    recall  f1-score   support

     chicken       0.94      0.95      0.95       172
        duck       0.97      0.96      0.97       310

    accuracy                           0.96       482
   macro avg       0.96      0.96      0.96       482
weighted avg       0.96      0.96      0.96       482

